# Practice Convolutional Neural Networks (CNN) and Transfer Learning

In [1]:
from keras import backend as K
K.set_image_dim_ordering('th')

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Activation

from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.preprocessing.image import (ImageDataGenerator,
                                       array_to_img,
                                       img_to_array,
                                       load_img)

import numpy as np
import pandas as pd
import os
import cv2 

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

Using TensorFlow backend.


Check what resources Tensorflow has to work with

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18100128700293186033
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 3879469056
locality {
  bus_id: 1
}
incarnation: 17093075520536632950
physical_device_desc: "device: 0, name: GRID K520, pci bus id: 0000:00:03.0"
]


__Make a Generator for make random images__

In [3]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')

Investigate random images

In [4]:
# img = load_img('band_data/train/bracelet/ZenithPilotBigDateSpecial032410401021m2410.jpg')  # this is a PIL image
# x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
# x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# # the .flow() command below generates batches of randomly transformed images
# # and saves the results to the `preview/` directory
# i = 0
# for batch in datagen.flow(x, batch_size=1,
#                           save_to_dir='preview', save_prefix='braclet', save_format='jpeg'):
#     i += 1
#     if i > 20:
#         break  # otherwise the generator would loop indefinitely

In [5]:
# img=mpimg.imread('preview/' + np.random.choice(os.listdir('preview')))
# imgplot = plt.imshow(img)

Get Rid of Previews

In [6]:
# ! rm preview/*.jpeg

__Function to Prepare Image__

In [7]:
def prepare_image(image_path):
    im = cv2.resize(cv2.imread(image_path), (224, 224)).astype(np.float32)

    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)
    return im

In [18]:
def preprocess_image(im):

    # 'RGB'->'BGR'
    im = im[:, :, ::-1]
    # Zero-center by mean pixel
    im[:, :, 0] -= 103.939
    im[:, :, 1] -= 116.779
    im[:, :, 2] -= 123.68
    return im

__Define the VGG_16 Model__

In [9]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3,3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3,3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3,3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3,3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3,3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3,3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [10]:
synset = pd.read_csv('synset_words.txt', skipinitialspace=True, names = ['synset', 'words'])
# model = VGG_16('vgg16_weights_tf_dim_ordering_tf_kernels.h5')
# sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(optimizer=sgd, loss='categorical_crossentropy')

### Transfer Learning

it turns out that the lower level featured learned by VGG16 on imagenet are still applicable to other problems with natural images. If we can preserve the lower-level features, we can just train a new model on those features. (In fact, in the case of 'softmax', we can think of this as just training a new multinomial logistic regression, on those convolution features)

Lets just snip off last layer.

In [11]:
model = VGG_16('vgg16_weights_tf_dim_ordering_tf_kernels.h5')
model.layers.pop() # removes the final layer

A Caveat

if we just add a new layer with default weights, it is going to be very wrong the first iteration. Since it is so wrong, the gradient will be huge, and because we are using back propagation those errors will be sent down stream into the lower level features. This can quickly destroy the rest of the network.

In order to retrain this model we must protect the lower-level features, until our new layers have reached more stability. We can do this by freezing those layers

In [12]:
# Freeze convolutional layers
for layer in model.layers:
    layer.trainable = False
    

now lets add our new layer. We can think of this as logistic regression with drop out for regularization

Then you would just train like normal

In [13]:
model.add(Dropout(0.5))
model.add(Dense(1))# must match the number of classes you are predicting
model.add(Activation('softmax'))
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
            loss='binary_crossentropy', metrics=['accuracy'])

Set Up Train/Test Image Generators

In [19]:
train_data_dir = 'band_data/train'
validation_data_dir = 'band_data/test'
img_height = 224
img_width = 224

In [20]:
nb_train_samples = 6000
nb_validation_samples = 2000
epochs = 50
batch_size = 8

In [21]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function=preprocess_image)

test_datagen = ImageDataGenerator(rescale=1./255,
                                  preprocessing_function=preprocess_image)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

Found 7025 images belonging to 2 classes.
Found 3461 images belonging to 2 classes.


In [ ]:
# fine-tune the model
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/50
750/750 [==============================] - 405s - loss: 6.7330 - acc: 0.5777 - val_loss: 6.9270 - val_acc: 0.5655
Epoch 2/50
750/750 [==============================] - 405s - loss: 6.9482 - acc: 0.5642 - val_loss: 7.0491 - val_acc: 0.5578
Epoch 3/50
333/750 [============>.................] - ETA: 169s - loss: 6.9060 - acc: 0.5668

In [ ]:
model.save_weights("model_test1.h5")